In [1]:
from tools.data_reading import CompressedWindowDataset, CocoWindowDataset
import torch
from torch import nn
from torchvision import transforms
from torch.utils.data import DataLoader
from tools import utils
from tools.engine import train_one_epoch, evaluate
from model_zoo.InputInjection import InputInjection
from torch.utils.tensorboard import SummaryWriter

In [2]:
dataset = CocoWindowDataset()
img, spectr, lab = dataset[0]

loading annotations into memory...
Done (t=0.05s)
creating index...


KeyError: 'image_id'

In [ ]:
dataset = CompressedWindowDataset()
img, spectr, lab = dataset[0]
# the ones with missing end labels are missing (just make code to fill labels!)

In [ ]:
for i in range(len(dataset)):
    x = dataset[i]

In [ ]:
train = DataLoader(dataset,batch_size = 1, shuffle=True, collate_fn=utils.collate_fn)

In [ ]:
img, spectr, lab = next(iter(train))
img = torch.unsqueeze(img[0], 0).float()
spectr = torch.unsqueeze(spectr[0], 0).float()

In [ ]:
print(img.shape)
print(spectr.shape)
print(lab)

In [ ]:
model = InputInjection()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'running on {device}')
if device == 'cuda':
    device_id = torch.cuda.current_device()
    print(f'using gpu {torch.cuda.get_device_name(device_id)}')
model.to(device)
writer = SummaryWriter()
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params,lr=0.00003)

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    train_one_epoch(writer, model, optimizer, train, device, epoch, print_freq=100)
    evaluate(writer, model, train, device, print_freq=100)
    torch.save(model.state_dict(), f'inputinjection-train-1.weights')